# Basketball Shots Analysis training notebook

## Installation and libraries

In [ ]:
%pip install ultralytics
%pip install roboflow
%pip install torch torchvision

## Importing necessary libraries

In [ ]:
from ultralytics import YOLO
from roboflow import Roboflow
import torch
from sys import platform
import os
import shutil

## Download the dataset

You can download the dataset from Roboflow [this link](https://universe.roboflow.com/cricket-qnb5l/basketball-xil7x/dataset/1). Make sure to unzip the file and place it in your working directory.

Or you can use the following code to download and unzip the dataset directly in your notebook:

In [ ]:
rf = Roboflow(api_key="Your_Roboflow_API_Key")
project = rf.workspace("cricket-qnb5l").project("basketball-xil7x")
version = project.version(1)
dataset = version.download("yolov8")

## Train the model

We will use a pre-trained model and fine-tune it on our basketball shots dataset using ultralytics YOLOv8.
We use yolov8m model for this task.

**Before training make sure you have a GPU enabled runtime to speed up the training process.**

The training script includes:
- Automatic device detection (CUDA/MPS/CPU)
- CUDA optimizations for faster training
- Automatic batch size detection based on GPU memory
- Optimized training parameters (AdamW optimizer, mixed precision, etc.)

In [ ]:
model_path = "yolov8m.pt"
epochs = 50
image_size = 640
batch_size = -1
workers = 8

if platform in ['linux', 'win32'] and torch.cuda.is_available():
    device = 'cuda'
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    torch.set_float32_matmul_precision('high')
    print(f'Using Device: {device}')
    print(f'   GPU: {torch.cuda.get_device_name(0)}')
    print(f'   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')
    print('CUDA optimizations enabled')
elif platform == 'darwin':
    device = 'mps'
    print(f'Using Device: {device} (Apple Silicon)')
else:
    device = 'cpu'
    print(f'Using Device: {device} (CPU - training will be slower)')

if batch_size == -1:
    if torch.cuda.is_available():
        vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
        if vram_gb >= 24:
            batch_size = 32
        elif vram_gb >= 12:
            batch_size = 16
        elif vram_gb >= 8:
            batch_size = 12
        else:
            batch_size = 8
        print(f'Auto-detected batch size: {batch_size} (VRAM: {vram_gb:.1f}GB)')
    else:
        batch_size = 8
        print(f'Using batch size: {batch_size} (CPU mode)')

model = YOLO(model_path)
print(f'Training {epochs} epochs with optimized settings')

training_args = {
    'data': dataset.location,
    'epochs': epochs,
    'imgsz': image_size,
    'device': device,
    'batch': batch_size,
    'workers': workers,
    'amp': True,
    'cache': True,
    'optimizer': 'AdamW',
    'cos_lr': True,
    'close_mosaic': 10,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 0.0,
    'translate': 0.1,
    'scale': 0.5,
    'shear': 0.0,
    'flipud': 0.0,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'patience': 50,
    'save': True,
    'save_period': 10,
    'plots': True,
    'verbose': True,
}

results = model.train(**training_args)

print('Training completed!')
print(f'Results saved in: runs/detect/train')
print(f'Best model: runs/detect/train/weights/best.pt')
print(f'Last checkpoint: runs/detect/train/weights/last.pt')

In [ ]:
best_model_path = 'runs/detect/train/weights/best.pt'
if os.path.exists(best_model_path):
    models_dir = '../models'
    os.makedirs(models_dir, exist_ok=True)
    destination = os.path.join(models_dir, 'shot.pt')
    shutil.copy(best_model_path, destination)
    print(f'Model copied to {destination}')
    print('You can now use this model in the application!')
else:
    print('Model not found. Make sure training has completed successfully.')


## Download and Use Trained Model

When the training is complete, the trained model will be saved in the `runs/detect/train/weights/` directory.

### Model Files

- **`best.pt`**: Best model based on validation metrics (recommended for production)
- **`last.pt`**: Last checkpoint from training

### Download from Notebook

1. In the left panel, navigate to the "Files" section
2. Go to `runs/detect/train/weights/`
3. Right-click on `best.pt` and select "Download"

### Copy to Models Directory

To use the model in the application, copy it to the `models/` directory:

```python
import shutil

# Copy best model to models directory
shutil.copy('runs/detect/train/weights/best.pt', '../models/shot.pt')
print('Model copied to models/shot.pt')
```

### Alternative: Use Python Script

You can also use the provided training script `yolo_cuda_trainer.py`:

```bash
python yolo_cuda_trainer.py -d data.yaml -e 50 -i 640 -b 16
```

Or with Docker:

```bash
docker build -t basketball-trainer .
docker run --gpus all -v $(pwd)/runs:/app/runs basketball-trainer
```